In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

import string

import re

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/atsukosakurai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/atsukosakurai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/atsukosakurai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# import data
df = pd.read_csv('2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv')
print(df.shape)
df.head()

(40623, 11)


,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
3,4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm


In [3]:
df.isnull().sum()

Unnamed: 0            0
id                    0
text                  0
label                 0
type                  0
model_wrong       14526
db.model_preds    14526
status                0
round                 0
split                 0
annotator             0
dtype: int64

In [4]:
df.duplicated().sum()

0

In [5]:
df = df.dropna()
df.shape

(26097, 11)

In [6]:
df['label'].value_counts()

hate       15253
nothate    10844
Name: label, dtype: int64

In [7]:
# hate speech detected scenario 1:
# label == hate AND model_wrong == False
cond1a = df['label'] == 'hate'
cond1b = df['model_wrong'] == False
print(len(df[cond1a & cond1b]))
df[cond1a & cond1b].head(3)

5441


,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
9,10,10009,I don't work this hard so that all those pakis...,hate,notgiven,False,0.09288,dynabench entry,1,train,cagnluiznm
10,11,10010,I don't work this hard so that all those musli...,hate,notgiven,False,0.46144,dynabench entry,1,train,cagnluiznm


In [8]:
# hate speech detected scenario 2:
# label == nothate AND model_wrong == True
cond2a = df['label'] == 'nothate'
cond2b = df['model_wrong'] == True
print(len(df[cond2a & cond2b]))
df[cond2a & cond2b].head(3)

5624


,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
28,29,10029,I dont hatefucking Islam,nothate,none,True,0.05814,dynabench entry,1,train,vde7gn0nrl
29,30,10030,I dont hate fucking Muslims,nothate,none,True,0.00707,dynabench entry,1,train,vde7gn0nrl
31,32,10032,Government is incompetent,nothate,none,True,0.06605,dynabench entry,1,train,vde7gn0nrl


In [10]:
# print sample of first 10 texts labeled "correctly" as hate:

for t in df[cond1a & cond1b]['text'][:5]:
    print(t)
    print('\n')
    
for t in df[cond2a & cond2b]['text'][:5]:
    print(t)
    print('\n')

dalits are lowlife scum


I don't work this hard so that all those pakis can take the benefits


I don't work this hard so that all those muslims can take the benefits


Why is it that Africans smell weird? Do they not wash?


Arabs are dumb


I dont hatefucking Islam


I dont hate fucking Muslims


Government is incompetent


short women are fucking cute


disabled people are stupid




In [9]:
# add a new column to simplify dataset:
# new column = "hate_detected"
# will be true if:
# label = 'hate' AND model_wrong = false OR
# label = 'nothate' AND model_wrong = true

df['hate_detected'] = np.where((cond1a & cond1b)|(cond2a & cond2b), True, False)

In [10]:
df.head()

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator,hate_detected
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3,False
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm,True
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm,False
3,4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3,False
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm,False


In [11]:
df['hate_detected'].value_counts()

False    15032
True     11065
Name: hate_detected, dtype: int64

In [12]:
# sanity check
len(df[cond2a & cond2b]) + len(df[cond1a & cond1b])

11065

In [13]:
# get rid of all columns other than text and hate_detected
df.columns
df = df.drop(columns=['Unnamed: 0', 'id', 'type', 'model_wrong',
       'db.model_preds', 'status', 'round', 'split', 'annotator','label'])

In [14]:
df.head()

,text,hate_detected
0,It should be legal for a husband to rape his wife,False
1,dalits are lowlife scum,True
2,dalits are lowlives,False
3,It was a better world when women dare not ques...,False
4,I don't work this hard so that those immigrant...,False


In [15]:
# lowercase all words:
def make_lower(a_string):
    return a_string.lower()

# test it
my_sent = 'ALL CAPS AND NOTHING ELSE HERE'
make_lower(my_sent)

'all caps and nothing else here'

In [16]:
# remove all punctuation

def remove_punc(a_string):
    a_string = re.sub(r'[^\w\s]', '', a_string)
    return a_string

my_sent = 'I_purposely! left @a lot+of=punctuations here!!%%%'
remove_punc(my_sent)

'I_purposely left a lotofpunctuations here'

In [17]:
# remove all stopwords

def remove_stopwords(a_string):
    words = word_tokenize(a_string)
    valid_words = []
    for word in words:
        if word not in stopwords:
            valid_words.append(word)
    a_string = ' '.join(valid_words)
    return a_string

my_sent = 'This is an apple, and I picked it from that tree on top of the hill next to the ocean'
remove_stopwords(my_sent)

'This apple , I picked tree top hill next ocean'

In [18]:
# stem words

def stem_words(a_string):
    porter = PorterStemmer()
    words = word_tokenize(a_string)
    valid_words = []
    
    for word in words:
        stemmed_word = porter.stem(word)
        valid_words.append(stemmed_word)
        
    a_string = ' '.join(valid_words)
    
    return a_string

my_sent = 'I am reading this book and then swimming to Mars.'
stem_words(my_sent)

'I am read thi book and then swim to mar .'

In [20]:
# pipeline function:
def text_pipeline(input_string):
    input_string = make_lower(input_string)
    input_string = remove_punc(input_string)
    input_string = remove_stopwords(input_string)
    return input_string

df['text_clean'] = df['text'].apply(make_lower)
print('Original:', df['text'][0])
print('Cleaned:', df['text_clean'][0])

Original: It should be legal for a husband to rape his wife
Cleaned: it should be legal for a husband to rape his wife


In [21]:
# look at "new" df
df.head()

,text,hate_detected,text_clean
0,It should be legal for a husband to rape his wife,False,it should be legal for a husband to rape his wife
1,dalits are lowlife scum,True,dalits are lowlife scum
2,dalits are lowlives,False,dalits are lowlives
3,It was a better world when women dare not ques...,False,it was a better world when women dare not ques...
4,I don't work this hard so that those immigrant...,False,i don't work this hard so that those immigrant...


In [22]:
df['hate_detected'].value_counts()

False    15032
True     11065
Name: hate_detected, dtype: int64

In [23]:
# lower case, remove stops words, stem 

df['text_clean'] = df['text'].apply(make_lower)
print('Original:', df['text'][0])
print('Cleaned:', df['text_clean'][0])

Original: It should be legal for a husband to rape his wife
Cleaned: it should be legal for a husband to rape his wife


In [24]:
df.head()

,text,hate_detected,text_clean
0,It should be legal for a husband to rape his wife,False,it should be legal for a husband to rape his wife
1,dalits are lowlife scum,True,dalits are lowlife scum
2,dalits are lowlives,False,dalits are lowlives
3,It was a better world when women dare not ques...,False,it was a better world when women dare not ques...
4,I don't work this hard so that those immigrant...,False,i don't work this hard so that those immigrant...


In [39]:
# define variables:

X = df['text_clean'].values

y = df['hate_detected'].values

In [40]:
# create a tokenizer, configured only to take into account the 20 most common words
tokenizer = Tokenizer(num_words=20)

In [41]:
# build the word index
tokenizer.fit_on_texts(X)

In [42]:
# turn strings into lists of integer indices
sequences = tokenizer.texts_to_sequences(X)

In [43]:
# show the vocabulary
word_index = tokenizer.word_index
print(tokenizer.word_index)
print('Found %s unique tokens.' % len(word_index))

{'the': 1, 'i': 2, 'to': 3, 'a': 4, 'are': 5, 'and': 6, 'is': 7, 'of': 8, 'that': 9, 'in': 10, 'people': 11, 'you': 12, 'not': 13, 'it': 14, 'be': 15, 'they': 16, 'my': 17, 'this': 18, 'all': 19, 'for': 20, 'so': 21, 'with': 22, 'black': 23, 'have': 24, 'women': 25, 'fucking': 26, 'as': 27, 'do': 28, 'should': 29, 'just': 30, 'like': 31, 'me': 32, 'but': 33, 'on': 34, 'their': 35, 'them': 36, "don't": 37, 'if': 38, 'love': 39, 'we': 40, 'think': 41, 'there': 42, 'white': 43, 'about': 44, 'can': 45, 'no': 46, 'who': 47, 'how': 48, 'was': 49, 'hate': 50, 'would': 51, 'want': 52, 'when': 53, 'because': 54, 'at': 55, 'or': 56, 'out': 57, 'from': 58, 'get': 59, 'will': 60, 'why': 61, 'men': 62, 'what': 63, 'one': 64, 'everyone': 65, 'more': 66, 'country': 67, 'our': 68, 'your': 69, 'fuck': 70, 'up': 71, "it's": 72, "i'm": 73, 'by': 74, 'being': 75, 'muslims': 76, 'these': 77, 'shit': 78, 'she': 79, 'an': 80, 'know': 81, 'really': 82, 'he': 83, 'has': 84, 'than': 85, 'am': 86, 'only': 87, 'w

In [44]:
print(sequences)

[[14, 15, 4, 3], [5], [5], [14, 4, 13], [2, 18, 9, 19, 1], [2, 18, 9, 19, 1], [2, 18, 9, 19, 1], [2, 18, 9, 19, 1], [2, 18, 9, 19, 1], [2, 18, 9, 19, 1], [2, 18, 9, 19, 1], [5, 4, 3], [5, 4, 3], [5, 4, 3], [11, 5, 4, 3], [11, 5, 4], [7, 14, 9, 16, 13], [7, 14, 9, 11, 16, 13], [7, 14, 9, 11, 16, 13], [7, 14, 9, 11, 16, 13], [7, 14, 9, 11, 16, 13], [7, 14, 9, 16, 13], [7, 14, 9, 11, 16, 13], [7, 14, 9, 16, 13], [7, 14, 9, 16, 13], [7, 14, 9, 11, 16, 13], [2], [2], [2], [2], [1, 8, 7], [7], [11, 5, 13], [11, 5, 3], [11, 5], [5], [5], [5], [11, 5], [5], [7, 13], [5], [11, 5, 13], [5], [15], [15], [5], [5], [5], [11], [11, 5], [5], [5], [5, 16, 3, 18], [11, 5, 13], [11], [11], [6, 2, 3], [5], [11], [11], [5], [15, 12], [12, 4], [4], [], [], [3], [7, 4], [4], [13, 4], [], [13, 4], [15], [3, 1], [], [], [], [1, 3], [3, 3], [], [1], [8, 1], [1, 3, 6], [5, 3, 4], [3, 19], [3], [], [], [5], [5], [15], [1, 16], [2, 17, 4, 4, 8], [17], [17], [7], [7], [7], [7], [7], [7], [7], [7], [1, 7], [2, 12, 

In [45]:
# identify max length 
max_length = 0
for review_number in range(len(sequences)):
    numberOfWords = len(sequences[review_number])
    if(numberOfWords) > (max_length):
        max_length = numberOfWords
print(max_length)

108


In [46]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [47]:
import numpy as np

In [48]:
# padding the sequence for shorter reviews
data = pad_sequences(sequences, maxlen=max_length)
y = np.asarray(y)
print(data)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

[[ 0  0  0 ... 15  4  3]
 [ 0  0  0 ...  0  0  5]
 [ 0  0  0 ...  0  0  5]
 ...
 [ 0  0  0 ...  5  8  1]
 [ 0  0  0 ...  0  0  5]
 [ 0  0  0 ...  5  3 15]]
Shape of data tensor: (26097, 108)
Shape of label tensor: (26097,)


In [49]:
# load word embeddings
import gensim

In [51]:
# load this giant google corpus as kvps
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [57]:
unique_words = len(word_index)
total_words = unique_words + 1
skipped_words = 0
dimension = 300
embedding_matrix = np.zeros((total_words, dimension))


In [58]:
for word, index in tokenizer.word_index.items():
    try:
        embedding_vector = wv[word]
    except:
        skipped_words = skipped_words + 1
        pass
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
print('Enbedding Matrix Shape:', embedding_matrix.shape)

Enbedding Matrix Shape: (18567, 300)


In [59]:
from tensorflow.keras.layers import *

In [60]:
embedding_layer = Embedding(total_words,
                           dimension,
                           weights=[embedding_matrix],
                           input_length=max_length,
                           trainable=False)

In [62]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [68]:
# define the model
model = Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(128,activation='relu',
                   return_sequences=True))
model.add(SimpleRNN(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 108, 300)          5570100   
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 108, 128)          54912     
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 256)               98560     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 5,723,829
Trainable params: 153,729
Non-trainable params: 5,570,100
_________________________________________________________________
None


In [70]:
from tensorflow.keras.optimizers import *

In [74]:
opt = SGD(lr=0.01,decay=1e-6)
model.compile(loss='binary_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])
model.fit(data, y, epochs=10, verbose=1)

Epoch 1/10
816/816 [==============================] - 27s 32ms/step - loss: 0.6833 - accuracy: 0.5541
Epoch 2/10
816/816 [==============================] - 26s 31ms/step - loss: 0.6692 - accuracy: 0.6007
Epoch 3/10
816/816 [==============================] - 26s 31ms/step - loss: 0.6605 - accuracy: 0.6104
Epoch 4/10
816/816 [==============================] - 26s 32ms/step - loss: 0.6548 - accuracy: 0.6101
Epoch 5/10
816/816 [==============================] - 26s 32ms/step - loss: 0.6512 - accuracy: 0.6125
Epoch 6/10
816/816 [==============================] - 26s 32ms/step - loss: 0.6492 - accuracy: 0.6103
Epoch 7/10
816/816 [==============================] - 26s 32ms/step - loss: 0.6454 - accuracy: 0.6156
Epoch 8/10
816/816 [==============================] - 26s 31ms/step - loss: 0.6466 - accuracy: 0.6183
Epoch 9/10
816/816 [==============================] - 26s 32ms/step - loss: 0.6412 - accuracy: 0.6227
Epoch 10/10
816/816 [==============================] - 26s 32ms/step - loss: 0.642